# Todo9  
Margaret Jones (mmj32)  
Tutorial: http://www.pitt.edu/~naraehan/presentation/Movie+Reviews+sentiment+analysis+with+Scikit-Learn.html  
17 October 2017

## sklearn

In [61]:
#Let's get these imports though
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.datasets import load_files

#you need these for the multinomial stuff
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

# for confusion matrix
from sklearn.metrics import confusion_matrix

In [4]:
#Turn off pretty printing of jupyter notebook... it generates long lines
%pprint

Pretty printing has been turned OFF


In [5]:
#Show all outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [22]:
#md is movie directory
md = "/Users/Margaret/Documents/Data_Science/movie_reviews"

In [23]:
#loading all files as training data. 
movie_train = load_files(md, shuffle=True)

In [35]:
#check the length of the data, there should be 2000 data
len(movie_train)
len(movie_train.data)
len(movie_train.filenames)

5

2000

2000

In [26]:
# check target names, there should be 2, pos and neg
movie_train.target_names

['neg', 'pos']

In [28]:
#Let's take a loot at that first file
movie_train.data[0][:500]

b"arnold schwarzenegger has been an icon for action enthusiasts , since the late 80's , but lately his films have been very sloppy and the one-liners are getting worse . \nit's hard seeing arnold as mr . freeze in batman and robin , especially when he says tons of ice jokes , but hey he got 15 million , what's it matter to him ? \nonce again arnold has signed to do another expensive blockbuster , that can't compare with the likes of the terminator series , true lies and even eraser . \nin this so cal"

In [32]:
dir(movie_train)

['DESCR', 'data', 'filenames', 'target', 'target_names']

In [29]:
#okay, now to check the filepath
movie_train.filenames[0]

'/Users/Margaret/Documents/Data_Science/movie_reviews/neg/cv405_21868.txt'

In [36]:
#targeeeet pos or neg?
movie_train.target[0]
movie_train.target[1]
movie_train.target[2]
# 0 is negative, and 1 is positive

0

1

1

In [38]:
#Says positive and negative instead of 0 and 1
movie_train.target_names[0]
movie_train.target_names[1]

'neg'

'pos'

## Count Vectorizer and TF-IDF

In [46]:
# import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# convert freq counts into TF-IDF
# (Term Frequency - Inverse Document Frequency)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
sents_tfidf = tfidf_transformer.fit_transform(sents_counts)

In [40]:
sents = ['A rose is a rose is a rose is a rose.',
         'Oh, what a fine day it is.',
         "You haven't seen nothing yet!"]

In [43]:
#initialize the CountVectorizer
foovec = CountVectorizer(min_df=1, tokenizer=nltk.word_tokenize)
sents_counts = foovec.fit_transform(sents)

#unique words are mapped to indices
foovec.vocabulary_

{'a': 3, 'rose': 12, 'is': 7, '.': 2, 'oh': 11, ',': 1, 'what': 14, 'fine': 5, 'day': 4, 'it': 8, 'you': 16, 'have': 6, "n't": 9, 'seen': 13, 'nothing': 10, 'yet': 15, '!': 0}

In [44]:
#(doc_count, #_unique_words)
sents_counts.shape

(3, 17)

In [45]:
#how about those vectors in an array?
sents_counts.toarray()

array([[0, 0, 1, 4, 0, 0, 0, 3, 0, 0, 0, 0, 4, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1]], dtype=int64)

In [47]:
#TF-IDF values in a fancy array
sents_tfidf.toarray()

array([[ 0.        ,  0.        ,  0.13650997,  0.54603988,  0.        ,
         0.        ,  0.        ,  0.40952991,  0.        ,  0.        ,
         0.        ,  0.        ,  0.71797683,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.35955412,  0.27345018,  0.27345018,  0.35955412,
         0.35955412,  0.        ,  0.27345018,  0.35955412,  0.        ,
         0.        ,  0.35955412,  0.        ,  0.        ,  0.35955412,
         0.        ,  0.        ],
       [ 0.37796447,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.37796447,  0.        ,  0.        ,  0.37796447,
         0.37796447,  0.        ,  0.        ,  0.37796447,  0.        ,
         0.37796447,  0.37796447]])

## Back to Those Movie Reviews

In [51]:
# initialize movie vector object
movie_vec = CountVectorizer(min_df=2, tokenizer=nltk.word_tokenize)
movie_counts = movie_vec.fit_transform(movie_train.data)

In [52]:
#test searching to see how many times some words appear
print(movie_vec.vocabulary_.get('screen'))
print(movie_vec.vocabulary_.get('seagal'))

19637
19690


In [53]:
#shape
movie_counts.shape

(2000, 25313)

In [54]:
#converting into TF-IDF values
tfidf_transformer = TfidfTransformer()
movie_tfidf = tfidf_transformer.fit_transform(movie_counts)

In [55]:
#checking the dimensions again to make sure I didn't screw up
movie_tfidf.shape

(2000, 25313)

In [57]:
#split this data into training and test sets
docs_train, docs_test, y_train, y_test = train_test_split(movie_tfidf, movie_train.target, test_size = 0.20, 
    random_state = 12)

In [58]:
# train
clf = MultinomialNB().fit(docs_train, y_train)

In [59]:
y_pred = clf.predict(docs_test)
sklearn.metrics.accuracy_score(y_test, y_pred)

0.82250000000000001

In [62]:
#Making that confusion matrix tho
cm = confusion_matrix(y_test, y_pred)
cm

array([[176,  30],
       [ 41, 153]])

### Blurb  
I didn't take LING1330 but I think all of this is really interesting. I get a little lost with the last little bit here but the lecture in class helped a bit. I think it would be more interesting if this data wasnt weighted with equally positive and negative reviews. I feel like there is more to be done if there was more realistic data. 